#                       Capstone Project for DataScience
##       The segmenting and clustering of the neighborhoods in 
##                               the City of Toronto, Canada
##                                           Part3
## Segmented, visualized, clustered the combined DataFrame formed by 
## scraping BeatifuSoup, augmented with their latitude and longitude 
## information, using FourSquare APIs, Folium, and KMeans
###                                         Krishna Yellamilli 06/30/2020

Let's start off by importing the necessary basic libraries to get compile the web info

In [1]:
import numpy as np # For mult-dimensional arrays usage

import pandas as pd # For DataFrames usage

import requests # Enough for handling web requests with REST APIs
import urllib.request # Needed for free https sites including WikiPedia
from urllib.request import urlopen # Needed for free https sites including WikiPedia

import json #For handling json file
from pandas.io.json import json_normalize # to transform json into pd.DataFrame

from bs4 import BeautifulSoup # for processing html docs

import geocoder # for getting the geolocation information from a post code
from geopy.geocoders import Nominatim # for getting the geolocation from an address

import matplotlib.cm as cm # for color maps
import matplotlib.colors as colors # for converting numbers and color arguments to RGB and RGBA schemes

from sklearn.cluster import KMeans # for clustering KMeans module

# Ensured FourSquare APIs are available

import folium # for maps

print("Importing all necessary modules/libraries completed.")

Importing all necessary modules/libraries completed.


Let's remove any limitations that hinder our interactive work just in case

In [2]:
pd.set_option('display.max_columns', None) # Removing display limitattion on columns
pd.set_option('display.max_rows', None) # Removing display limitattion on rows

Let's start get rolling by getting the info from the Wikipedia on Toronto's neighborhood info using the provided web page 

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')
#print(soup.prettify()) for debug purpose

Let's prepare the DataFrame infrastcture starting with the table column headers

In [4]:
#initializing the columns
columns = []

for tag in soup.find_all('th'): # Looping through the table column headers
    if (tag.string): # To avoid Null tag.srings
        columns.append(tag.string.rstrip())
columns

['Postal Code', 'Borough', 'Neighborhood']

Let's gather the table data in the order of post code, burough, and grouped neighborhoods

In [5]:
pc_hood = {} # There can be more than one neighborhoold in a post code
pc_brh = {} # The burough associated with the post code
i = 0 # initializing to keep track of the table rows with data
for tag in soup.find_all('td'): # Looping through the table data
    i_mod = i % 3
    if i_mod == 0:
        post_code = tag.string
    elif i_mod == 1:
        brh = tag.string
    elif i_mod == 2:
        if post_code and (brh.rstrip().upper() != 'NOT ASSIGNED'): # ignoring entries with a borough "Not Assigned"
            brh=brh.rstrip()
            nbrhood = tag.string.rstrip()
            post_code = post_code.rstrip()
            if nbrhood.upper() == 'NOT ASSIGNED': nbrhood = brh
            if post_code in pc_hood.keys():
                pc_hood[post_code] = pc_hood[post_code]+", " + nbrhood
            else: pc_hood[post_code] = nbrhood
            if post_code not in pc_brh.keys():
                pc_brh[post_code] = brh
            else: # Checking to see if a post code ends up in two different buroughs
                print("ERROR: post code {} is already associated with {} burough".format(post_code, pc_brh[post_code]))
                print("       post code {} is now being associated with {} burough".format(post_code, brh))
    i += 1 # preparing to index the next table entry
    
if (i % 3 != 0): #Error checking for conistency of processing 3 columns per row in table
    print("The table columns is not a multiple of 3")
    print("{}".format(i))

pc_lists =[] # Initializing the list
#for key in sorted(pc_brh.keys()): # Populating the list with processed table rows in the post code order for debug
for key in pc_brh.keys(): # Populating the list with processed table rows in the keys order
    pc_lists.append([key, pc_brh[key], pc_hood[key]])

T_DF=pd.DataFrame(pc_lists, columns=columns) # Initializing the dataframe
T_DF

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [6]:
# Checking the size of the Toronot post code DataFrame size
T_DF.shape

(103, 3)

Gathering the latitude and longitudes for the post codes

In [7]:
#Turning this code to markdown as 20 times LIMIT 
pc_lat = {} # for storing postal code's lattitude
pc_lng = {} # for storing postal code's longitude
pcs = 0
for pc in pc_brh.keys():
    lat_lng = None # Initial state
    i = 0 # Iteratio monitoring
    LIMIT = 20 # limit setting for the number of iterations
    while (lat_lng is None):
        g = geocoder.google('{} Toronto, Ontario, Canada'.format(pc)) # Using google module
        lat_lng = g.latlng
        if i > LIMIT: 
            print("Exceeded {} tries for the postal code {} look up with idex of {}.".format(i+1, pc, pcs))
            break
        else: i += 1
    if (lat_lng is not None):
        pc_lat[pc] = lat_lng[0]
        pc_lng[pc] = lat_lng[1]
    pcs += 1
for pc in pc_lat.keys():
    print ("{} {} {} {} {}".format(key, pc_brh[pc], pc_hood[pc], pc_lat[pc], pc_lng[pc]))
        

Exceeded 22 tries for the postal code M3A look up with idex of 0.
Exceeded 22 tries for the postal code M4A look up with idex of 1.
Exceeded 22 tries for the postal code M5A look up with idex of 2.
Exceeded 22 tries for the postal code M6A look up with idex of 3.
Exceeded 22 tries for the postal code M7A look up with idex of 4.
Exceeded 22 tries for the postal code M9A look up with idex of 5.
Exceeded 22 tries for the postal code M1B look up with idex of 6.
Exceeded 22 tries for the postal code M3B look up with idex of 7.
Exceeded 22 tries for the postal code M4B look up with idex of 8.
Exceeded 22 tries for the postal code M5B look up with idex of 9.
Exceeded 22 tries for the postal code M6B look up with idex of 10.
Exceeded 22 tries for the postal code M9B look up with idex of 11.
Exceeded 22 tries for the postal code M1C look up with idex of 12.
Exceeded 22 tries for the postal code M3C look up with idex of 13.
Exceeded 22 tries for the postal code M4C look up with idex of 14.
Excee

Multiple tries did not work with Google at different times on different days; hence used the provided csv data.

In [8]:
pc_lat_lng = pd.read_csv("C:\\Users\\krish\\DataScience\\projects\\capstone\\Geospatial_Coordinates.csv", header=0)
pc_lat_lng.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
# Merging both DFs
T_DF=T_DF.join(pc_lat_lng.set_index('Postal Code'), on='Postal Code')
T_DF

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [10]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent = 'Toronto_Exploration')
location = geolocator.geocode(address)

Tt_map = folium.Map(location=[location.latitude, location.longitude], zoom_start = 10)

In [11]:
#setting up popup info for exploring differnt points of the city
for pc, brh, lat, lng in zip(T_DF['Postal Code'], T_DF['Borough'], T_DF['Latitude'], T_DF['Longitude']):
    str_lbl = "{}, {}".format(brh, pc)
    flm_lbl = folium.Popup(str_lbl, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = flm_lbl,
        color = 'blue',
        fill=True,
        fill_color= '#3186cc',
        fill_opacity = 0.7,
        parse_html=False).add_to(Tt_map)

Tt_map

In [12]:
#Getting prepared for Foursquare API
CLIENT_ID = 'HZY0G2MR2KO4FX2EKZOQNHPB5XY4KLBCXGGMCEE4KOWUPCH2' # your Foursquare ID
CLIENT_SECRET = 'QOZNAJVN04LSW0PT4LWLFU3EXKBX05DLTMHEXEYDI0IWJWWL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=100 # Maximum number of results
radius=500 # Radius of search

In [13]:
#Set up to examine each postal code area
def getNearbyVenuesDF(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print("processing {} ".format(name))
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        # making the GET request
        # results = requests.get(url).json()["response"]['venues'][0]['items']
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude',
            'Venue Longitude', 'Venue Category']
    return(nearby_venues)

In [14]:
Tt_venues_DF = getNearbyVenuesDF(names=T_DF['Postal Code'], latitudes=T_DF['Latitude'], longitudes=T_DF['Longitude'])

processing M3A 
processing M4A 
processing M5A 
processing M6A 
processing M7A 
processing M9A 
processing M1B 
processing M3B 
processing M4B 
processing M5B 
processing M6B 
processing M9B 
processing M1C 
processing M3C 
processing M4C 
processing M5C 
processing M6C 
processing M9C 
processing M1E 
processing M4E 
processing M5E 
processing M6E 
processing M1G 
processing M4G 
processing M5G 
processing M6G 
processing M1H 
processing M2H 
processing M3H 
processing M4H 
processing M5H 
processing M6H 
processing M1J 
processing M2J 
processing M3J 
processing M4J 
processing M5J 
processing M6J 
processing M1K 
processing M2K 
processing M3K 
processing M4K 
processing M5K 
processing M6K 
processing M1L 
processing M2L 
processing M3L 
processing M4L 
processing M5L 
processing M6L 
processing M9L 
processing M1M 
processing M2M 
processing M3M 
processing M4M 
processing M5M 
processing M6M 
processing M9M 
processing M1N 
processing M2N 
processing M3N 
processing M4N 
processi

In [15]:
Tt_venues_DF.head()

,Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,M4A,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,M4A,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [16]:
Tt_venues_DF.shape

(2127, 7)

In [19]:
#Creating one hot DF
T_venues_1HotDF = pd.get_dummies(Tt_venues_DF[['Venue Category']], prefix='', prefix_sep='')
T_venues_1HotDF.insert(loc=0, column='Postal Code', value=Tt_venues_DF['Postal Code'])#USED like an INDEX!
#T_venues_1HotDF.shape # shape: (2127, 269)
T_venues_1HotDF.head()

,Postal Code,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M3A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [21]:
T_venues_Grouped = T_venues_1HotDF.groupby('Postal Code').mean() # The Postal Code becomes index now
#T_venues_Grouped.shape # shape: (98, 268)
T_venues_Grouped.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
Postal Code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [22]:
T_venues_Grouped.reset_index(inplace=True) # To get the default index by returning the index to column
T_venues_Grouped.head()

,Postal Code,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    if ind < 3:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    else:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postal Code'] = T_venues_Grouped['Postal Code']
for ind in np.arange(T_venues_Grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(T_venues_Grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Fast Food Restaurant,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner
1,M1C,Construction & Landscaping,Bar,Yoga Studio,Drugstore,Discount Store
2,M1E,Intersection,Electronics Store,Breakfast Spot,Restaurant,Medical Center
3,M1G,Coffee Shop,Korean Restaurant,Yoga Studio,Donut Shop,Diner
4,M1H,Caribbean Restaurant,Gas Station,Bank,Athletics & Sports,Thai Restaurant


In [24]:
# Clustering into 5
kclusters = 5
Tt_grouped_clustering = T_venues_Grouped.drop(labels='Postal Code', axis=1) # the same as T_venues_Grouped.drop('Postal Code', 1)
Tt_grouped_clustering.shape # is (98 268)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Tt_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[:]

array([0, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 4, 2, 2, 1, 2, 1, 4, 1,
       4, 2, 1, 1, 1, 2, 2, 3, 2, 1, 2, 2, 1, 1, 2, 4, 1, 1, 1, 4, 2, 1,
       1, 2, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1,
       1, 1, 1, 1, 2, 4, 2, 2, 2, 1, 4, 0, 2, 2, 2, 1, 1, 1, 2, 2, 1, 2,
       3, 2, 1, 2, 3, 4, 1, 4, 2, 2])

In [25]:
#Inserting cluster label
neighborhoods_venues_sorted.insert(loc=1, column='Cluster Label', value=kmeans.labels_)
neighborhoods_venues_sorted.head()

,Postal Code,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,0,Fast Food Restaurant,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner
1,M1C,2,Construction & Landscaping,Bar,Yoga Studio,Drugstore,Discount Store
2,M1E,2,Intersection,Electronics Store,Breakfast Spot,Restaurant,Medical Center
3,M1G,1,Coffee Shop,Korean Restaurant,Yoga Studio,Donut Shop,Diner
4,M1H,2,Caribbean Restaurant,Gas Station,Bank,Athletics & Sports,Thai Restaurant


In [27]:
Tt_pc_clustered_venues = pc_lat_lng.join(neighborhoods_venues_sorted.set_index('Postal Code'), on='Postal Code')
Tt_pc_clustered_venues.shape

(103, 9)

In [29]:
#Noticed that that we did not get venues for all postal codes; hence, we need to drop the NaN that got populated in the above DF
Tt_pc_clustered_venues.dropna(inplace=True)
# Tt_pc_clustered_venues.shape # (98, 9)
Tt_pc_clustered_venues.head()

,Postal Code,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,43.806686,-79.194353,0.0,Fast Food Restaurant,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner
1,M1C,43.784535,-79.160497,2.0,Construction & Landscaping,Bar,Yoga Studio,Drugstore,Discount Store
2,M1E,43.763573,-79.188711,2.0,Intersection,Electronics Store,Breakfast Spot,Restaurant,Medical Center
3,M1G,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Yoga Studio,Donut Shop,Diner
4,M1H,43.773136,-79.239476,2.0,Caribbean Restaurant,Gas Station,Bank,Athletics & Sports,Thai Restaurant


In [30]:
# Cluster label was changed to float in Tt_pc_clusters_venues; hence change back to int32
Tt_pc_clustered_venues = Tt_pc_clustered_venues.astype({'Cluster Label':'int32'})

Tt_pc_clustered_venues
#Tt_pc_clustered_venues.shape # (98 9) the OLD indices numvbers are maintained

,Postal Code,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,43.806686,-79.194353,0,Fast Food Restaurant,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner
1,M1C,43.784535,-79.160497,2,Construction & Landscaping,Bar,Yoga Studio,Drugstore,Discount Store
2,M1E,43.763573,-79.188711,2,Intersection,Electronics Store,Breakfast Spot,Restaurant,Medical Center
3,M1G,43.770992,-79.216917,1,Coffee Shop,Korean Restaurant,Yoga Studio,Donut Shop,Diner
4,M1H,43.773136,-79.239476,2,Caribbean Restaurant,Gas Station,Bank,Athletics & Sports,Thai Restaurant
5,M1J,43.744734,-79.239476,2,Playground,Jewelry Store,Yoga Studio,Doner Restaurant,Dim Sum Restaurant
6,M1K,43.727929,-79.262029,1,Discount Store,Coffee Shop,Chinese Restaurant,Department Store,Yoga Studio
7,M1L,43.711112,-79.284577,2,Bus Line,Bakery,Park,Intersection,Ice Cream Shop
8,M1M,43.716316,-79.239476,1,Motel,American Restaurant,Dim Sum Restaurant,Diner,Discount Store
9,M1N,43.692657,-79.264848,2,College Stadium,Skating Rink,General Entertainment,Café,Eastern European Restaurant


In [31]:
# Setting up color schemes
# set color scheme for the clusters
x = np.arange(kclusters) # array([0, 1, 2, 3, 4])
ys = [i + x + (i*x)**2 for i in range(kclusters)] # List of 5 arrays of which are manipulated array([0, 1, 2, 3, 4])
colors_array = cm.rainbow(np.linspace(0, 1, len(ys))) # shape of (5,4)
rainbow = [colors.rgb2hex(i) for i in colors_array]  # ['#8000ff', '#00b5eb', '#80ffb4', '#ffb360', '#ff0000']


In [32]:
Tt_clusters_map = folium.Map(location=[location.latitude, location.longitude], zoom_start = 10)
for pc, clstr, lat, lng in zip(Tt_pc_clustered_venues['Postal Code'], Tt_pc_clustered_venues['Cluster Label'], Tt_pc_clustered_venues['Latitude'], Tt_pc_clustered_venues['Longitude']):
    str_lbl = "{} cluster {}".format(pc,clstr)
    flm_lbl = folium.Popup(str_lbl, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = flm_lbl,
        color = rainbow[clstr],
        fill=True,
        fill_color= rainbow[clstr],
        fill_opacity = 0.7,
        parse_html=False).add_to(Tt_clusters_map)
Tt_clusters_map

In [33]:
# Cluster 1:
Tt_pc_clustered_venues[Tt_pc_clustered_venues['Cluster Label'] == 0]

,Postal Code,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,43.806686,-79.194353,0,Fast Food Restaurant,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner
80,M6M,43.691116,-79.476013,0,Fast Food Restaurant,Sandwich Place,Discount Store,Bar,Dog Run


Cluster 1 is sparsely populated area, not much to do around!

In [34]:
# Cluster 2:
Tt_pc_clustered_venues[Tt_pc_clustered_venues['Cluster Label'] == 1]


,Postal Code,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,M1G,43.770992,-79.216917,1,Coffee Shop,Korean Restaurant,Yoga Studio,Donut Shop,Diner
6,M1K,43.727929,-79.262029,1,Discount Store,Coffee Shop,Chinese Restaurant,Department Store,Yoga Studio
8,M1M,43.716316,-79.239476,1,Motel,American Restaurant,Dim Sum Restaurant,Diner,Discount Store
18,M2J,43.778517,-79.346556,1,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Japanese Restaurant
22,M2N,43.770120,-79.408493,1,Ramen Restaurant,Sushi Restaurant,Sandwich Place,Japanese Restaurant,Pizza Place
24,M2R,43.782736,-79.442259,1,Pizza Place,Bank,Coffee Shop,Discount Store,Pharmacy
27,M3C,43.725900,-79.340923,1,Restaurant,Gym,Asian Restaurant,Coffee Shop,Beer Store
28,M3H,43.754328,-79.442259,1,Coffee Shop,Bank,Diner,Supermarket,Mobile Phone Shop
29,M3J,43.767980,-79.487262,1,Falafel Restaurant,Coffee Shop,Caribbean Restaurant,Bar,Furniture / Home Store
34,M4A,43.725882,-79.315572,1,Coffee Shop,Portuguese Restaurant,Hockey Arena,Intersection,Yoga Studio


Cluster 2 has plenty of coffee shops in downtown area setting

In [35]:
# Cluster 3:
Tt_pc_clustered_venues[Tt_pc_clustered_venues['Cluster Label'] == 2]

,Postal Code,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,M1C,43.784535,-79.160497,2,Construction & Landscaping,Bar,Yoga Studio,Drugstore,Discount Store
2,M1E,43.763573,-79.188711,2,Intersection,Electronics Store,Breakfast Spot,Restaurant,Medical Center
4,M1H,43.773136,-79.239476,2,Caribbean Restaurant,Gas Station,Bank,Athletics & Sports,Thai Restaurant
5,M1J,43.744734,-79.239476,2,Playground,Jewelry Store,Yoga Studio,Doner Restaurant,Dim Sum Restaurant
7,M1L,43.711112,-79.284577,2,Bus Line,Bakery,Park,Intersection,Ice Cream Shop
9,M1N,43.692657,-79.264848,2,College Stadium,Skating Rink,General Entertainment,Café,Eastern European Restaurant
10,M1P,43.757410,-79.273304,2,Indian Restaurant,Pet Store,Light Rail Station,Vietnamese Restaurant,Chinese Restaurant
11,M1R,43.750072,-79.295849,2,Bakery,Smoke Shop,Breakfast Spot,Middle Eastern Restaurant,Yoga Studio
12,M1S,43.794200,-79.262029,2,Lounge,Breakfast Spot,Skating Rink,Latin American Restaurant,Clothing Store
13,M1T,43.781638,-79.304302,2,Pizza Place,Pharmacy,Gas Station,Shopping Mall,Fast Food Restaurant


Cluster 3 has plenty of shopping and dining venues spread across the city

In [36]:
# Cluster 4:
Tt_pc_clustered_venues[Tt_pc_clustered_venues['Cluster Label'] == 3]

,Postal Code,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
32,M3M,43.728496,-79.495697,3,Food Truck,Baseball Field,Donut Shop,Diner,Discount Store
91,M8Y,43.636258,-79.498509,3,Baseball Field,Yoga Studio,Donut Shop,Diner,Discount Store
97,M9M,43.724766,-79.532242,3,Baseball Field,Yoga Studio,Donut Shop,Diner,Discount Store


Cluster 4 is popular for outdoor sport due basefield

In [99]:
# Cluster 5:
Tt_pc_clustered_venues[Tt_pc_clustered_venues['Cluster Label'] == 4]

,Postal Code,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
14,M1V,43.815252,-79.284577,4,Playground,Park,Yoga Studio,Doner Restaurant,Dessert Shop
23,M2P,43.752758,-79.400049,4,Convenience Store,Park,Donut Shop,Dim Sum Restaurant,Diner
25,M3A,43.753259,-79.329656,4,Food & Drink Shop,Park,Yoga Studio,Dim Sum Restaurant,Diner
40,M4J,43.685347,-79.338106,4,Park,Pizza Place,Convenience Store,Event Space,Ethiopian Restaurant
44,M4N,43.728020,-79.388790,4,Park,Bus Line,Swim School,Yoga Studio,Doner Restaurant
50,M4W,43.679563,-79.377529,4,Park,Playground,Trail,Yoga Studio,Dog Run
74,M6E,43.689026,-79.453512,4,Park,Women's Store,Pool,Doner Restaurant,Dessert Shop
79,M6L,43.713756,-79.490074,4,Bakery,Park,Construction & Landscaping,Basketball Court,Yoga Studio
98,M9N,43.706876,-79.518188,4,Convenience Store,Park,Donut Shop,Dim Sum Restaurant,Diner
100,M9R,43.688905,-79.554724,4,Sandwich Place,Park,Mobile Phone Shop,Yoga Studio,Dog Run


Cluster 5 is popular for outdoor activities due to abundant parks